# Loop 2 Analysis: Understanding the Optimization Landscape

## Key Findings from Previous Experiments

1. **Baseline at local optimum**: bbox3 ran 80 rounds with ZERO improvement
2. **Ensemble from 40 snapshots**: Achieved 70.559 (0.1 better than 70.676)
3. **Target**: 68.919154 (gap: 1.64 points from best ensemble)

## Critical Insights from Discussions
- Symmetric solutions (42 upvotes) vs Asymmetric solutions (34 upvotes) debate
- The solution is at a very tight local optimum
- Need fundamentally different approaches

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

# Tree geometry
def get_tree_polygon(cx, cy, angle):
    trunk_w, trunk_h = 0.15, 0.2
    base_w, mid_w, top_w = 0.7, 0.4, 0.25
    tip_y, tier_1_y, tier_2_y, base_y = 0.8, 0.5, 0.25, 0.0
    trunk_bottom_y = -trunk_h
    
    vertices = [
        (0.0, tip_y), (top_w/2, tier_1_y), (top_w/4, tier_1_y),
        (mid_w/2, tier_2_y), (mid_w/4, tier_2_y), (base_w/2, base_y),
        (trunk_w/2, base_y), (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y), (-trunk_w/2, base_y),
        (-base_w/2, base_y), (-mid_w/4, tier_2_y), (-mid_w/2, tier_2_y),
        (-top_w/4, tier_1_y), (-top_w/2, tier_1_y)
    ]
    poly = Polygon(vertices)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, xoff=cx, yoff=cy)
    return poly

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

print('Functions defined')

In [ ]:
# Load the ensemble solution and analyze per-N scores
def analyze_per_n_scores(csv_path):
    df = pd.read_csv(csv_path)
    scores = []
    
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        rows = df[df['id'].str.startswith(prefix)]
        if len(rows) != n:
            continue
        
        all_points = []
        for _, row in rows.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = get_tree_polygon(x, y, deg)
            coords = np.array(poly.exterior.coords)
            all_points.extend(coords)
        
        all_points = np.array(all_points)
        min_xy = all_points.min(axis=0)
        max_xy = all_points.max(axis=0)
        side = max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])
        score_n = (side ** 2) / n
        scores.append({'n': n, 'side': side, 'score': score_n, 'efficiency': n / (side ** 2)})
    
    return pd.DataFrame(scores)

# Analyze the ensemble solution
ensemble_df = analyze_per_n_scores('/home/code/ensemble_70.559.csv')
print(f'Total score: {ensemble_df["score"].sum():.6f}')
print(f'\nTop 10 worst efficiency (highest score contribution):')
print(ensemble_df.nsmallest(10, 'efficiency')[['n', 'side', 'score', 'efficiency']])

In [ ]:
# Compare with baseline
baseline_df = analyze_per_n_scores('/home/code/current.csv')
print(f'Baseline total score: {baseline_df["score"].sum():.6f}')
print(f'Ensemble total score: {ensemble_df["score"].sum():.6f}')
print(f'Improvement: {baseline_df["score"].sum() - ensemble_df["score"].sum():.6f}')

# Find N values where ensemble is better
merged = baseline_df.merge(ensemble_df, on='n', suffixes=('_base', '_ens'))
merged['improvement'] = merged['score_base'] - merged['score_ens']
print(f'\nN values where ensemble is better:')
print(merged[merged['improvement'] > 0.001][['n', 'score_base', 'score_ens', 'improvement']].head(20))

In [ ]:
# Find all CSV files across all snapshots
import subprocess

result = subprocess.run(
    ['find', '/home/nonroot/snapshots/santa-2025', '-name', '*.csv', '-type', 'f'],
    capture_output=True, text=True
)
csv_files = [f for f in result.stdout.strip().split('\n') if f]
print(f'Found {len(csv_files)} CSV files across all snapshots')

In [ ]:
# Check for overlaps in the ensemble solution
def has_overlap(trees):
    if len(trees) <= 1:
        return False
    tree_index = STRtree(trees)
    for i, poly in enumerate(trees):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(trees[idx]) and not poly.touches(trees[idx]):
                intersection = poly.intersection(trees[idx])
                if intersection.area > 1e-10:
                    return True
    return False

def validate_csv(csv_path):
    df = pd.read_csv(csv_path)
    overlaps = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        rows = df[df['id'].str.startswith(prefix)]
        if len(rows) != n:
            continue
        
        trees = []
        for _, row in rows.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            trees.append(get_tree_polygon(x, y, deg))
        
        if has_overlap(trees):
            overlaps.append(n)
    return overlaps

# Validate ensemble
overlaps = validate_csv('/home/code/ensemble_70.559.csv')
if overlaps:
    print(f'WARNING: Overlaps found in N={overlaps}')
else:
    print('No overlaps found - solution is valid!')